In [1]:
%load_ext autoreload
%autoreload 2

import os


import sys

import scipy
import scipy.io
import time

import os.path
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_WARNINGS"] = "FALSE"


import tensorflow as tf
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

CHECKPOINT_PATH = "/home/vlq26735/code/PhD/GradientPathologiesPINNs/checkpoints/1DPoisson/"
MODE = "ntk"

src_path = os.path.abspath(os.path.join('../../'))
print(f"src_path= {src_path}")

if src_path not in sys.path:
    sys.path.insert(0, src_path)


from src.cases.Poisson import ntk



src_path= /home/vlq26735/code/PhD/GradientPathologiesPINNs


In [2]:
class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name=None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name

    def sample(self, N):
        x = self.coords[0:1, :] + (self.coords[1:2, :] - self.coords[0:1, :]) * np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y

In [3]:
# Define solution and its Laplace
a = 4

def u(x, a):
  return np.sin(np.pi * a * x)

def u_xx(x, a):
  return -(np.pi * a)**2 * np.sin(np.pi * a * x)

In [4]:
# Define computional domain
bc1_coords = np.array([[0.0], [0.0]])
bc2_coords = np.array([[1.0], [1.0]])
dom_coords = np.array([[0.0], [1.0]])

# Training data on u(x) -- Dirichlet boundary conditions

nn  = 100

X_bc1 = dom_coords[0, 0] * np.ones((nn // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((nn // 2, 1))
X_u = np.vstack([X_bc1, X_bc2])
Y_u = u(X_u, a)

X_r = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
Y_r = u_xx(X_r, a)

nn = 1000
X_star = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
u_star = u(X_star, a)
r_star = u_xx(X_star, a)

nIter =40001
bcbatch_size = 500
ubatch_size = 5000
mbbatch_size = 128



# Define model
mode = 'ntk'
layers = [1, 500,500, 1]
activFun = "tanh"
method = "mini-batch"

iterations = 1
starter_learning_rate = 1.0e-5

# Create residual sampler
gpu_options = tf.GPUOptions(visible_device_list="0")
tf.reset_default_graph()
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=False, log_device_placement=False))
# with  as sess:

model = ntk.Possion_ntk(layers, X_u, Y_u, X_r, Y_r , mode , starter_learning_rate, CHECKPOINT_PATH , sess)    


model.print("Using mode: " , model.mode)
model.print("neural network: " , model.layers )
model.print("Bc1 Training data size : " ,X_bc1.shape[0])
model.print("Bc2 Training data size : " ,X_bc2.shape[0])
model.print("interior Training data size : " ,X_r.shape[0])

model.print("Activation function: " , activFun)
model.print("number of iterations: " , nIter)
model.print("starter_learning_rate: " , starter_learning_rate)

model.print("Method desciption : gradual learing rate , with  " , method ," batch. ")


2024-02-28 07:24:28.529163: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 07:24:28.648337: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2445400000 Hz
2024-02-28 07:24:28.648670: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561b335aaf90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-28 07:24:28.648694: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-02-28 07:24:28.648880: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Using mode: ntk
neural network: [1, 500, 500, 1]
Bc1 Training data size : 50
Bc2 Training data size : 50
interior Training data size : 100
Activation function: tanh
number of iterations: 40001
starter_learning_rate: 1.0000e-05
Method desciption : gradual learing rate , with  mini-batch batch. 


In [5]:

# Train model
start_time = time.time()

print("mini_batch method is used")
model.trainmb(nIter, mbbatch_size)
elapsed = time.time() - start_time

# Predictions
u_pred = model.predict_u(X_star)
r_pred = model.predict_r(X_star)
# Predictions

error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_r = np.linalg.norm(r_star - r_pred, 2) / np.linalg.norm(r_star, 2)

model.print('elapsed: {:.2e}'.format(elapsed))

model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_r: {:.2e}'.format(error_r))


model.print('mean value of lambda_bc: {:.2e}'.format(np.average(model.adaptive_constant_bcs_log)))
model.print('first value of lambda_bc: {:.2e}'.format(model.adaptive_constant_bcs_log[0]))
model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_v: {:.2e}'.format(error_r))

model.save_NN()
model.plot_grad()
model.plot_lambda()
model.plt_prediction( X_star , u_star , u_pred)
# sess.close()  
    

mini_batch method is used
It: 0, Loss: 7.805e+03, Loss_bcs: 4.763e-01, Loss_res: 7.804e+03 ,Time: 13.41


mean_grad_bcs: 1.4757e-01
mean_grad_res: 1.0063e+02


Compute NTK...


lam_res: 1.0047e+00
lam_bc: 2.1386e+02
Gradients information stored ...


It: 100, Loss: 1.821e+02, Loss_bcs: 3.720e+01, Loss_res: 1.449e+02 ,Time: 71.25


...

It: 39100, Loss: 1.798e-02, Loss_bcs: 2.797e-05, Loss_res: 1.795e-02 ,Time: 24.16
It: 39200, Loss: 1.789e-02, Loss_bcs: 2.776e-05, Loss_res: 1.787e-02 ,Time: 48.26
It: 39300, Loss: 1.779e-02, Loss_bcs: 2.760e-05, Loss_res: 1.776e-02 ,Time: 71.94
It: 39400, Loss: 1.770e-02, Loss_bcs: 2.742e-05, Loss_res: 1.768e-02 ,Time: 96.14
It: 39500, Loss: 1.764e-02, Loss_bcs: 2.723e-05, Loss_res: 1.761e-02 ,Time: 120.49
It: 39600, Loss: 1.756e-02, Loss_bcs: 2.703e-05, Loss_res: 1.753e-02 ,Time: 144.44
It: 39700, Loss: 1.747e-02, Loss_bcs: 2.689e-05, Loss_res: 1.745e-02 ,Time: 168.02
It: 39800, Loss: 1.740e-02, Loss_bcs: 2.673e-05, Loss_res: 1.737e-02 ,Time: 192.45
It: 39900, Loss: 1.733e-02, Loss_bcs: 2.658e-05, Loss_res: 1.731e-02 ,Time: 215.99


Gradients information stored ...
mean_grad_bcs: 1.4073e-03
mean_grad_res: 7.4371e-02


It: 40000, Loss: 1.726e-02, Loss_bcs: 2.641e-05, Loss_res: 1.723e-02 ,Time: 269.95
Compute NTK...


lam_res: 1.0009e+00
lam_bc: 1.0801e+03
elapsed: 9.39e+03
Relative L2 error_u: 3.85e-03
Relative L2 error_r: 1.13e-03
mean value of lambda_bc: nan


IndexError: list index out of range

In [ ]:
Relative L2 error_u: 5.63e-02
Relative L2 error_v: 4.55e-02